In [1]:
# pip install transformers datasets scikit-learn torch

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("datasets/cleanedDataSecondModel_merged.csv")
#remame
df = df[['quote', 'category_encoded']]
df.columns = ['text', 'label']

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)


In [ ]:
from transformers import DistilBertTokenizerFast

#tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)


In [4]:
import torch

class QuoteDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            key: torch.tensor(val[idx])
            for key, val in self.encodings.items()
        } | {'labels': torch.tensor(self.labels[idx])}

train_dataset = QuoteDataset(train_encodings, train_labels)
test_dataset = QuoteDataset(test_encodings, test_labels)


In [5]:
# pip install transformers[torch]


In [6]:
# pip install accelerate>=0.26.0

In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

num_labels = len(set(df['label']))
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="no",
    logging_dir="./logs",
    logging_steps=10
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()


In [8]:
import numpy as np
from sklearn.metrics import classification_report

preds = trainer.predict(test_dataset)
pred_labels = np.argmax(preds.predictions, axis=1)

print(classification_report(test_labels, pred_labels))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         2
           2       1.00      1.00      1.00         1
           3       1.00      0.50      0.67         2
           4       0.00      0.00      0.00         3
           5       0.00      0.00      0.00         1
           6       0.14      0.50      0.22         6
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         1
           9       0.21      0.75      0.33         4
          12       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         1
          15       0.00      0.00      0.00         1
          16       0.14      1.00      0.25         1
          17       0.00      0.00      0.00         1
          18       0.00      0.00      0.00         1
          20       0.50      0.75      0.60         4
          21       0.00    

c:\Marsha\College Apps\Python\environments\nlp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Marsha\College Apps\Python\environments\nlp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Marsha\College Apps\Python\environments\nlp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

In [ ]:
#predict
test_quote = "Success is not final, failure is not fatal"
inputs = tokenizer(test_quote, return_tensors="pt", truncation=True, padding=True, max_length=128)

model.eval()

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=1).item()

print(f"\nPredicted label for: \"{test_quote}\" → {predicted_label}")



Predicted label for: "Success is not final, failure is not fatal" → 77


77 -> optimism